In [1]:
import numpy as np 
import pandas as pd


In [2]:
df = pd.read_csv("/Users/andreferdinand/Desktop/MOPT/MK/gym-MK/gym_MK/envs/Products.csv")

In [3]:
df

,PRODUCT_ID,PRODUCT,STORAGE,ENDPRODUCT,MIN,PROCESS_00,PROCESS_01,DEMAND_DIST,DIST_PARA,PROCESS_02
0,0,DOUGH,10,False,100,BREAD,BUNS,NaN,NaN,NaN
1,1,BREAD,10,True,70,NaN,NaN,NORMAL,"(1, 0.1)",NaN
2,2,BUNS,10,False,70,BUNS1,BUNS2,NaN,NaN,BUNS3
3,3,BUNS1,10,True,10,NaN,NaN,NORMAL,"(1, 0.1)",NaN
4,4,BUNS2,10,True,10,NaN,NaN,NORMAL,"(1, 0.1)",NaN
5,5,BUNS3,10,True,10,NaN,NaN,NORMAL,"(1, 0.1)",NaN


In [4]:
from helper_stats import min_prod, state
min_product = min_prod()
game_state =state()

In [5]:
game_state

In [6]:
from MK_env2 import MKEnv

In [7]:
game = MKEnv()

In [8]:
print(game.game_state.DEMAND_BREAD)
print(game.game_state.STORAGE_BREAD)
print(game.game_state_length)

0
10
10


In [9]:
game.action_length

4

In [10]:
game._get_demand()

In [12]:
game.reset()

In [11]:
game.game_state.DEMAND_BREAD

0.8821927079581877

In [12]:
game.demand_dist_end_para.DDISTPARA_BUNS2

(1, 0.1)

In [13]:
type(game.demand_dist_end_para.DDISTPARA_BREAD)

tuple

In [14]:
game.demand_dist_end.DDIST_BREAD

'NORMAL'

In [15]:
game.demand_dist_end_para.DDISTPARA_BREAD

(1, 0.1)

In [16]:
getattr(game.demand_dist_end,"DDIST_BREAD")

'NORMAL'

In [17]:
game.game_state.DEMAND_BREAD

0.8821927079581877

In [18]:
all([False,True])

False

In [19]:
game._get_reward()

-300

In [20]:
game.LAMBDA

5